In [1]:
import pathlib
import logging
import threading
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
np.random.seed(0)
from sklearn.model_selection import StratifiedKFold
import time
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
print(tf.__version__)
QuantasThreadsFinalizaram = 0
exibirRodando = False
anonimo = True
FearValidaComHL2 = True # Ao validar usa na mesma categoria de HL2
soHL2 = False # Ao treina o modelo do Fear deleta instancias de Spelunky
comHora = True

2.7.0


In [2]:
dispositivoUsado = '/cpu:0' 
#dispositivoUsado = '/gpu:0'


dados = pd.read_csv('dados-Ambos-Tudo.csv')

#QoE agora entre 0 e 6
dados["QoE"] = dados["QoE"] + 3


#DROPANDO LINHAS COM USO DE CPU E RAM ALTO
dados = dados[dados['UsoCpu'] < 81]
dados = dados[dados['UsoRam'] < 81]
numeroInstancias = len(dados[dados['Jogo'] != 'FEAR'])

#Modificand a data e hora de jogo para ser somente a hora
if(comHora):
    dados['TerminoPartida'] = dados['TerminoPartida'].str.split('-')
    dados['TerminoPartida'] = dados['TerminoPartida'].str[3]

#Embaralha os dados

    
#DROPANDO DADOS NÃO UTILIZADOS PELO MODELO
print(numeroInstancias)

if(comHora):
    dados.drop(['TempoJogo','UsoCpu', 'UsoRam', 'UsoGpu', 'UsoVram'], axis=1, inplace=True)
else:
    dados.drop(['TempoJogo','UsoCpu', 'UsoRam', 'UsoGpu', 'UsoVram','TerminoPartida'], axis=1, inplace=True)
dados.dropna(inplace=True)
dados = dados.sample(frac=1).reset_index(drop=True)
porcentagemTeste = 0.2
porcentagemValidacao = 0.1
NumeroCamadasInternas = 64

2020


In [12]:
#Arredondar os valores de um vetor (X.4 -> X e X.5 -> X+1)
def arredondaVetor(vetor):
  with tf.device(dispositivoUsado):
    novoVetor = []
    for x in range(len(vetor)):
        if(vetor[x]>=0):
            floatTeste = vetor[x] - int(vetor[x])
            if(floatTeste>=0.5):
                novoVetor.append(int(vetor[x])+1)
            else:
                novoVetor.append(int(vetor[x]))
        else:
            valor = vetor[x]*(-1.0)
            floatTeste = valor - int(valor)
            if(floatTeste>=0.5):
                novoVetor.append((int(valor)+1)*-1)
            else:
                novoVetor.append(int(valor)*-1)
    return novoVetor

In [13]:
#Arredondar os valores de um vetor (Compara vetor1 com 2 e retorna Float entre 0 e 100)
def porcentagemAcerto(vetor1,vetor2):
  with tf.device(dispositivoUsado):
    tamanho = float(len(vetor1))
    acertos = float(0)
    for x in range(len(vetor1)):
        if(vetor1[x]==vetor2[x]):
            acertos = acertos + 1.0
    acertos = (acertos/tamanho)*100.0
    return acertos

In [15]:
def rodarModeloRF(baseDados):
    n_estimators = 46
    max_features = 6
    max_depth = 9
    global porcentagemTeste
    tempDados = baseDados.copy(deep=True)
    tempDados = tempDados[tempDados['Jogo'] != 'FEAR']
    tempDados["NOME"].unique()
    tempDados["Jogo"].unique()
    dummies_jogador = pd.get_dummies(tempDados["NOME"])
    dummies_jogo = pd.get_dummies(tempDados["Jogo"])
    tempDados.drop(["NOME", "Jogo"], axis=1, inplace=True)
    tempDados = tempDados.join(dummies_jogador)
    tempDados = tempDados.join(dummies_jogo)
    if(comHora):
        X = np.asarray(tempDados[['DelayVideo', 'DelayComandos', 'JitterVideo', 'JitterComandos', 
                              'PerdaVideo', 'PerdaComandos','TerminoPartida']])
    else:
        X = np.asarray(tempDados[['DelayVideo', 'DelayComandos', 'JitterVideo', 'JitterComandos', 
                              'PerdaVideo', 'PerdaComandos']])
    y = np.asarray(tempDados['QoE'])
    scaler = StandardScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    X_temp = np.zeros((tempDados.shape[0],tempDados.shape[1]))
    X_temp[:,:X.shape[1]] = X
    X = X_temp
    if(comHora):
        X[:, 7:-1] = np.asarray(tempDados[list(tempDados.drop(['DelayVideo', 'DelayComandos', 'JitterVideo',
                    'JitterComandos', 'PerdaVideo', 'PerdaComandos','TerminoPartida','QoE'], axis=1))])
    else:
        X[:, 6:-1] = np.asarray(tempDados[list(tempDados.drop(['DelayVideo', 'DelayComandos', 'JitterVideo',
                    'JitterComandos', 'PerdaVideo', 'PerdaComandos','QoE'], axis=1))])
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=porcentagemTeste, random_state=1)
    X_train = np.asarray(X_train).astype(np.float32)
    X_test = np.asarray(X_test).astype(np.float32)
    y_train = np.asarray(y_train).astype(np.float32)
    y_test = np.asarray(y_test).astype(np.float32)
    
    
    from sklearn.metrics import mean_squared_error
    modelos = list()
    mses = list()
    acertos = list()
    XT = list()
    YT = list()
    kFold = StratifiedKFold(n_splits=10)
    for train, test in kFold.split(X_train, y_train):
        rfr=RandomForestRegressor(n_estimators=n_estimators,max_features=max_features,criterion='mse',bootstrap=True,max_depth=max_depth)
        rfr.fit(X_train[train],y_train[train])
        y_pred=rfr.predict(X_train[test])#X_test
        test_predictions_round = arredondaVetor(y_pred)
        porcentagemAcertos = porcentagemAcerto(y_train[test],test_predictions_round)#y_test
        #print("Acerto:",round(porcentagemAcertos,4))
        mse = mean_squared_error(y_train[test],test_predictions_round)#y_test
        #print("MSE: "+str(mse))
        mses.append(mse)
        modelos.append(rfr)
        acertos.append(porcentagemAcertos)
        XT.append(X_train[test])
        YT.append(y_train[test])
    menorMSE = 99
    iMSE = 0
    for i in range(len(mses)):
        if(mses[i]<menorMSE):
            menorMSE = mses[i]
            iMSE = i
    model = modelos[iMSE]
    mse = mses[iMSE]
    acerto = acertos[iMSE]
    #print("Menor MSE: "+str(mse)+" com acerto: "+str(acerto))
    y_pred = model.predict(X_test)
    test_predictions_round = arredondaVetor(y_pred)
    porcentagemAcertos1 = porcentagemAcerto(y_test,test_predictions_round)
    #print("Acerto com todos dados:",round(porcentagemAcertos1,4))
    mse1 = mean_squared_error(y_test,test_predictions_round)
    #print("MSE com todos dados: "+str(mse1))
        
    return mse,acerto,model,X_test,y_test

In [16]:
def rodarModeloRFcomFear(baseDados):
    n_estimators = 46
    max_features = 6
    max_depth = 9
    global porcentagemTeste
    tempDados = baseDados.copy(deep=True)
    if(soHL2):
        tempDados = tempDados[tempDados['Jogo'] != 'Spelunky']
    tempDados["NOME"].unique()
    tempDados["Jogo"].unique()
    dummies_jogador = pd.get_dummies(tempDados["NOME"])
    dummies_jogo = pd.get_dummies(tempDados["Jogo"])
    
    tempDados.drop(["NOME", "Jogo"], axis=1, inplace=True)
    
    tempDados = tempDados.join(dummies_jogador)
    tempDados = tempDados.join(dummies_jogo)
    
    dadosFear = tempDados[tempDados['FEAR'] == 1]
    tempDados = tempDados[tempDados['FEAR'] != 1]
    
    if(FearValidaComHL2):
        tempDados.drop('FEAR', axis=1, inplace=True)
        dadosFear['Half-Life 2'] = 1
        dadosFear.drop('FEAR', axis=1, inplace=True)

    if(comHora):
        X = np.asarray(tempDados[['DelayVideo', 'DelayComandos', 'JitterVideo', 'JitterComandos', 
                              'PerdaVideo', 'PerdaComandos','TerminoPartida']])
    
        XFear = np.asarray(dadosFear[['DelayVideo', 'DelayComandos', 'JitterVideo', 'JitterComandos', 
                              'PerdaVideo', 'PerdaComandos','TerminoPartida']])
    else:
        X = np.asarray(tempDados[['DelayVideo', 'DelayComandos', 'JitterVideo', 'JitterComandos', 
                              'PerdaVideo', 'PerdaComandos']])
    
        XFear = np.asarray(dadosFear[['DelayVideo', 'DelayComandos', 'JitterVideo', 'JitterComandos', 
                              'PerdaVideo', 'PerdaComandos']])
    
    y = np.asarray(tempDados['QoE'])
    
    yFear = np.asarray(dadosFear['QoE'])
    
    scaler = StandardScaler()
    
    scalerFear = StandardScaler()
    
    scaler.fit(X)
    
    scalerFear.fit(XFear)
    
    X = scaler.transform(X)
    
    XFear = scalerFear.transform(XFear)
    
    X_temp = np.zeros((tempDados.shape[0],tempDados.shape[1]))
    
    X_tempFear = np.zeros((dadosFear.shape[0],dadosFear.shape[1]))
    
    X_temp[:,:X.shape[1]] = X
    
    X_tempFear[:,:XFear.shape[1]] = XFear
    
    X = X_temp
    
    XFear = X_tempFear
    
    if(comHora):
        X[:, 7:-1] = np.asarray(tempDados[list(tempDados.drop(['DelayVideo', 'DelayComandos', 'JitterVideo',
                    'JitterComandos', 'PerdaVideo', 'PerdaComandos','TerminoPartida','QoE'], axis=1))])
    
        XFear[:, 7:-1] = np.asarray(dadosFear[list(dadosFear.drop(['DelayVideo', 'DelayComandos', 'JitterVideo',
                    'JitterComandos', 'PerdaVideo', 'PerdaComandos','TerminoPartida','QoE'], axis=1))])
    else:
        X[:, 6:-1] = np.asarray(tempDados[list(tempDados.drop(['DelayVideo', 'DelayComandos', 'JitterVideo',
                    'JitterComandos', 'PerdaVideo', 'PerdaComandos','QoE'], axis=1))])
    
        XFear[:, 6:-1] = np.asarray(dadosFear[list(dadosFear.drop(['DelayVideo', 'DelayComandos', 'JitterVideo',
                    'JitterComandos', 'PerdaVideo', 'PerdaComandos','QoE'], axis=1))])
    
    
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=porcentagemTeste, random_state=1)
    
    #Não precisa disso pois o XFear e yFear já são de teste
    #X_trainFear, X_testFear, y_trainFear, y_testFear = train_test_split( XFear, yFear, test_size=1.0, random_state=1)
    XFear = np.asarray(XFear).astype(np.float32)
    yFear = np.asarray(yFear).astype(np.float32)
    
    X_train = np.asarray(X_train).astype(np.float32)
    X_test = np.asarray(X_test).astype(np.float32)
    y_train = np.asarray(y_train).astype(np.float32)
    y_test = np.asarray(y_test).astype(np.float32)
    
    
    from sklearn.metrics import mean_squared_error
    modelos = list()
    mses = list()
    acertos = list()
    kFold = StratifiedKFold(n_splits=10)
    for train, test in kFold.split(X_train,y_train):
        rfr=RandomForestRegressor(n_estimators=n_estimators,max_features=max_features,criterion='mse',bootstrap=True,max_depth=max_depth)
        rfr.fit(X_train[train],y_train[train])
        y_pred=rfr.predict(X_train[test])#X_test
        test_predictions_round = arredondaVetor(y_pred)
        porcentagemAcertos = porcentagemAcerto(y_train[test],test_predictions_round)#y_test
        #print("Acerto:",round(porcentagemAcertos,4))
        mse = mean_squared_error(y_train[test],test_predictions_round)#y_test
        #print("MSE: "+str(mse))
        mses.append(mse)
        modelos.append(rfr)
        acertos.append(porcentagemAcertos)
    menorMSE = 99
    iMSE = 0
    for i in range(len(mses)):
        if(mses[i]<menorMSE):
            menorMSE = mses[i]
            iMSE = i
    model = modelos[iMSE]
    mse = mses[iMSE]
    acerto = acertos[iMSE]
    #print("Menor MSE: "+str(mse)+" com acerto: "+str(acerto))
    y_pred = model.predict(XFear)
    test_predictions_round = arredondaVetor(y_pred)
    porcentagemAcertos1 = porcentagemAcerto(yFear,test_predictions_round)
    #print("Acerto com todos dados:",round(porcentagemAcertos1,4))
    mse1 = mean_squared_error(yFear,test_predictions_round)
    #print("MSE com todos dados: "+str(mse1))
    
    return mse,acerto,model,XFear,yFear

In [18]:
def plot_heatMap(test_predictions,y_test,nomeHeatmap):
    import seaborn as sns
    test_predictions_round = arredondaVetor(test_predictions)
    x_axis_labels = ['E. Péssimo','Péssimo','Ruim','Regular','bom','Excelente','Ideal'] # labels for x-axis
    y_axis_labels = ['E. Péssimo','Péssimo','Ruim','Regular','bom','Excelente','Ideal'] # labels for y-axis
    
    heatMatrix = np.zeros((7,7))
    for x in range(len(y_test)):
        intX = int(test_predictions_round[x])
        intY = int(y_test[x])
        if(intX > 6):
            continue
        heatMatrix[intX][intY] = heatMatrix[intX][intY] +1
                
    sns.heatmap(heatMatrix, xticklabels=x_axis_labels, yticklabels=y_axis_labels, cmap="Greys")
    plt.savefig(nomeHeatmap+'.png', transparent=False)
    plt.show()

In [19]:
#Plota somente para os modelos TensorFlow
def plot_melhor(y_test, test_predictions,history,tipo):
  with tf.device(dispositivoUsado):
    
    histi = pd.DataFrame(history.history)
    histi['epoch'] = history.epoch

    plt.figure()
    plt.xlabel('Época')
    plt.ylabel('Erro Médio Absoluto [QoE]')
    plt.plot(histi['epoch'], histi['mae'],label='Erro de Treino')
    plt.plot(histi['epoch'], histi['val_mae'],label = 'Erro de Validação')
    plt.ylim([0,5])
    plt.legend()
    plt.savefig('melhor'+tipo+'comAcordoMAE.png')
    plt.show()
    
        
    plt.figure()
    plt.xlabel('Época')
    plt.ylabel('Erro Médio Quadratico [$QoE^2$]')
    plt.plot(histi['epoch'], histi['mse'],label='Erro de Treino')
    plt.plot(histi['epoch'], histi['val_mse'],label = 'Erro de Validação')
    plt.ylim([0,20])
    plt.legend()
    plt.savefig('melhor'+tipo+'comAcordoMSE.png')
    plt.show()
    
    
    plt.figure()
    plt.scatter(y_test, test_predictions)
    plt.xlabel('Valores Reais [QoE]')
    plt.ylabel('Predicões [QoE]')
    plt.axis('equal')
    plt.axis('square')
    plt.xlim([0,plt.xlim()[1]])
    plt.ylim([0,plt.ylim()[1]])
    _ = plt.plot([-100, 100], [-100, 100])
    plt.savefig('melhor'+tipo+'comAcordoPredicoesSemArredondar.png')
    plt.show()
    
    
    test_predictions_round = arredondaVetor(test_predictions)
    print("Porcentagem de acertos arredondado: "+str(porcentagemAcerto(y_test,test_predictions_round)))
    plt.figure()
    plt.scatter(y_test, test_predictions_round)
    plt.xlabel('Valores Reais [QoE]')
    plt.ylabel('Predicões Arredondadas [QoE]')
    plt.axis('equal')
    plt.axis('square')
    plt.xlim([0,plt.xlim()[1]])
    plt.ylim([0,plt.ylim()[1]])
    _ = plt.plot([-100, 100], [-100, 100])
    plt.savefig('melhor'+tipo+'comAcordoPredicoesArredondado.png')
    plt.show()
    
    plot_heatMap(test_predictions,y_test,"HeatMap_"+tipo)

In [24]:
randomForest = True
with tf.device(dispositivoUsado):
    dados2 = dados[dados['Jogo'] != 'FEAR']
        repeticoes = 30
        MSESem,AcertoSem,modeloSem,X_testSem,y_testSem = rodarModeloRF(dados2)
        for ABA in range(repeticoes):
            MSESem1,AcertoSem1,modeloSem1,X_testSem1,y_testSem1 = rodarModeloRF(dados2) #mse,acerto,model,X_test,y_test
            if(MSESem1<MSESem):
                MSESem = MSESem1
                AcertoSem = AcertoSem1
                modeloSem = modeloSem1
                X_testSem = X_testSem1
                y_testSem = y_testSem1
    
    print(MSESem)
    print(AcertoSem)
    print(modeloSem)
    print(X_testSem)
    print(y_testSem)

Teste completo Agrupamento e Fear, MSE e Acerto sem nada inicial:
MSE: 0.75 Acerto: 49.07



AttributeError: 'DataFrame' object has no attribute 'NOME'